In [ ]:
!pip install openai pandas



In [ ]:
from google.colab import userdata

# Получаем токен и url api
YANDEX_CLOUD_API_KEY  = userdata.get('YA_GPT_API_KEY')
YANDEX_CLOUD_FOLDER = "b1gp3qnjq6rhh31ttqqq"
ya_api_url= "https://llm.api.cloud.yandex.net/v1"

In [ ]:
#from yandex_chain import YandexLLM, ChatYandexGPT
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
#gpt = YandexLLM(folder_id=YANDEX_CLOUD_FOLDER, api_key=YANDEX_CLOUD_API_KEY)
#print(gpt('Привет! Придумай 10 новых слов для приветствия.'))

In [ ]:
# режим чата
#gpt = ChatYandexGPT(folder_id=YANDEX_CLOUD_FOLDER, api_key=YANDEX_CLOUD_API_KEY)
# print(gpt(
#     [
#         SystemMessage(content='Ты полезный ассистент'),
#         HumanMessage(content='Сколько планет в солнечной системе')
#     ]))

In [ ]:
# Установка необходимых библиотек (если нужно)

# Импорт
import os
import pandas as pd
import json
import re
from google.colab import files

In [ ]:
#загрузка csv
def upload_csv():
    """
    Загружает CSV-файл через интерфейс Colab.
    :return: Путь к загруженному файлу
    """
    print("Загрузите CSV-файл...")
    uploaded = files.upload()

    if not uploaded:
        raise FileNotFoundError("Файл не был загружен.")

    filename = list(uploaded.keys())[0]
    print(f"Файл '{filename}' успешно загружен.")
    return filename

def load_csv_to_dataframe(filepath):
    """
    Читает CSV-файл в pandas DataFrame.
    :param filepath: Путь к файлу
    :param encoding: Кодировка файла
    :return: DataFrame
    """
    try:
        df = pd.read_csv(filepath, on_bad_lines='skip', sep=None, engine='python')
        print(f"Файл успешно прочитан. Размер: {df.shape}")
        return df
    except Exception as e:
        raise RuntimeError(f"Ошибка при чтении CSV: {e}")

In [ ]:
# извлечение списка отзывов
def extract_reviews(df, text_column="review_text"):
    """
    Извлекает список отзывов из указанного столбца.
    :param df: DataFrame
    :param text_column: Название столбца с текстом отзыва
    :return: Список строк (отзывов)
    """
    if text_column not in df.columns:
        raise ValueError(f"Столбец '{text_column}' не найден в данных. Доступные: {list(df.columns)}")

    reviews = df[text_column].astype(str).fillna("").tolist()
    print(f"Извлечено {len(reviews)} отзывов из столбца '{text_column}'.")
    return reviews

In [ ]:
# обработка отзыва LLM
def call_llm_for_review(client, model, review, system_prompt):
    """
    Отправляет один отзыв в LLM и получает ответ.
    :param client: OpenAI-совместимый клиент
    :param model: Название модели
    :param review: Текст отзыва
    :param system_prompt: Системный промпт с инструкцией
    :return: Сырой строковый ответ от модели
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Распознай отзыв: {review}"}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"⚠️ Ошибка при обращении к LLM: {e}")
        return None

In [ ]:
def clean_and_parse_json(llm_response):
    """
    Очищает ответ LLM от markdown-разметки и парсит как JSON.
    Поддерживает случаи: ```json{...}```, `{...}`, с пробелами, переносами и т.п.
    :param llm_response: Строка от LLM
    :return: Распаршенный словарь или None
    """
    if not llm_response:
        return None

    try:
        # Удаляем возможные блоки markdown
        cleaned = re.sub(r'^\s*```(?:json)?|```\s*$', '', llm_response, flags=re.MULTILINE)
        cleaned = cleaned.strip()

        # Парсим JSON
        parsed = json.loads(cleaned)
        return parsed
    except json.JSONDecodeError as e:
        print(f"❌ Ошибка парсинга JSON: {e}")
        print(f"Сырой ответ: {repr(llm_response)}")
        return None

def flatten_results(data):
    """
    КОСТЫЛЬ убирает вложенность 'results', поднимая его содержимое на верхний уровень.

    Пример:
    {
      "review_id": 1,
      "results": { "annotations": [...] }
    }
    →
    {
      "review_id": 1,
      "annotations": [...]
    }
    """
    if isinstance(data, list):
        return [flatten_results(item) for item in data]
    elif isinstance(data, dict):
        # Если есть ключ 'results' — распаковываем его
        if "results" in data and isinstance(data["results"], dict):
            # Начинаем с копии ВЕРХНЕГО уровня (включая review_id)
            flattened = {k: v for k, v in data.items() if k != "results"}
            # Добавляем всё из results (перезаписывая только в случае конфликта)
            flattened.update(data["results"])
            return flattened
        else:
            # Рекурсивно обрабатываем вложенные структуры
            return {k: flatten_results(v) for k, v in data.items()}
    else:
        return data

In [ ]:
# создание результата
def build_final_entry(review_id, llm_parsed_response):
    """
    Создаёт один элемент результата.
    Берёт содержимое поля 'annotations' из ответа LLM и помещает в results.annotations.

    :param review_id: Индекс отзыва (int)
    :param llm_parsed_response: Распаршенный словарь от LLM (может быть None)
    :return: Словарь в формате {"review_id": ..., "results": {"annotations": [...]}}
    """
    # Если ответа нет — возвращаем пустой массив аннотаций
    if not llm_parsed_response:
        annotations_list = []
    else:
        # Извлекаем список из поля 'annotations', если оно есть
        if isinstance(llm_parsed_response, dict) and 'annotations' in llm_parsed_response:
            annotations_list = llm_parsed_response['annotations']
            # Проверяем, что это действительно список
            if not isinstance(annotations_list, list):
                print(f"⚠️ Ожидался список в 'annotations', получено: {type(annotations_list)}")
                annotations_list = []
        else:
            # Если структура другая (например, сразу категория + summary), оборачиваем в список
            # Например: {"category": "...", "summary": "..."} → превращаем в [{"category": "...", "summary": "..."}]
            if isinstance(llm_parsed_response, dict):
                annotations_list = [{k: v for k, v in llm_parsed_response.items() if k in ["category", "summary"]}]
                # На всякий случай проверим, что есть данные
                if not any(annotations_list[0].values()):
                    annotations_list = []
            else:
                annotations_list = []

    return {
        "review_id": review_id,
        "results": {
            "annotations": annotations_list
        }
    }

def save_results_to_json(data, output_filename="llm_annotations.json"):
    """
    Сохраняет результат в JSON-файл и предлагает скачать.
    :param data: Список словарей
    :param output_filename: Имя выходного файла
    """
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"💾 Результат сохранён в '{output_filename}'")

        # Для Colab — возможность скачать
        files.download(output_filename)
        print("📥 Файл доступен для скачивания.")
    except Exception as e:
        print(f"Ошибка при сохранении: {e}")

In [ ]:
def process_reviews_with_llm(client, model, reviews, system_prompt, batch_size=None):
    """
    Обрабатывает список отзывов через LLM.
    :param client: OpenAI-клиент
    :param model: Модель
    :param reviews: Список текстов отзывов
    :param system_prompt: Системный промпт
    :param batch_size: Опционально — ограничение на количество отзывов (для теста)
    :return: Список финальных словарей
    """
    results = []

    # Ограничиваем размер, если нужно (для тестирования)
    if batch_size:
        reviews = reviews[:batch_size]
        print(f"Обработка первых {len(reviews)} отзывов (режим теста).")

    total = len(reviews)
    print(f"Начинаем обработку {total} отзывов...")

    for idx, review in enumerate(reviews):
        if not review.strip():
            print(f"[{idx}] Пропущен пустой отзыв.")
            entry = build_final_entry(idx, None)
            results.append(entry)
            continue

        print(f"[{idx + 1}/{total}] Обработка отзыва...")
        raw_response = call_llm_for_review(client, model, review, system_prompt)

        if raw_response is None:
            print(f"[{idx}] Ошибка получения ответа от LLM.")
            entry = build_final_entry(idx, None)
        else:
            parsed = clean_and_parse_json(raw_response)
            if parsed is None:
                print(f"[{idx}] Не удалось распарсить ответ. Используется пустая аннотация.")
            entry = build_final_entry(idx, parsed)

        results.append(entry)

    print("✅ Все отзывы обработаны.")
    return results

In [ ]:
# чекпойнты
# def save_checkpoint(data, checkpoint_id):

#     filename = f"checkpoint_{checkpoint_id}.json"
#     with open(filename, 'w', encoding='utf-8') as f:
#         json.dump(data, f, ensure_ascii=False, indent=2)
#     print(f"✅ Чекпоинт {checkpoint_id} сохранён в '{filename}'")
#     files.download(filename)

def save_checkpoint(data, counter, prefix="checkpoint_"):
    """
    Сохраняет промежуточный результат.
    :param data: Список записей
    :param checkpoint_id: Номер чекпоинта
    """
    filename = f"{prefix}{counter}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"💾 Чекпоинт сохранён: {filename}")


def merge_checkpoints(output_file="final_result.json"):
    """
    Собирает все checkpoint_*.json в один файл.
    :param output_file: Имя финального файла
    """
    import glob

    checkpoints = sorted(glob.glob("checkpoint_*.json"))
    if not checkpoints:
        print("❌ Не найдено ни одного чекпоинта.")
        return

    final_data = []
    for cp in checkpoints:
        try:
            with open(cp, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if isinstance(data, list):
                    final_data.extend(data)
                else:
                    print(f"⚠️ Файл {cp} содержит несписковые данные, пропущено.")
        except Exception as e:
            print(f"❌ Ошибка чтения {cp}: {e}")

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(final_data, f, ensure_ascii=False, indent=2)

    print(f"🎯 Все чекпоинты объединены. Итоговый файл: {output_file}")
    files.download(output_file)


In [ ]:
# создание llm клиента
import openai
client = openai.OpenAI(
    api_key=YANDEX_CLOUD_API_KEY,
    base_url="https://llm.api.cloud.yandex.net/v1"
)
model_name = "llama"
model=f"gpt://{YANDEX_CLOUD_FOLDER}/{model_name}/latest"

response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "Ты очень умный ассистент."},
        {"role": "user", "content": "Что умеют большие языковые модели?"}
    ],
    temperature=0.3
)
print(response.choices[0].message.content)

Большие языковые модели, такие как я, умеют выполнять широкий спектр задач, связанных с обработкой и генерацией текста. Вот некоторые из вещей, которые мы умеем делать:

1. **Ответы на вопросы**: Мы можем понимать и отвечать на вопросы на различные темы, используя знания, которые мы получили во время обучения.
2. **Генерация текста**: Мы можем генерировать текст на основе заданного начала, темы или стиля. Это может быть полезно для написания статей, создания контента или даже генерации кода.
3. **Перевод текста**: Мы можем переводить текст с одного языка на другой, используя свои знания о лингвистических структурах и словарном запасе.
4. **Суммаризация текста**: Мы можем суммировать длинные тексты, выделяя основные моменты и суть информации.
5. **Классификация текста**: Мы можем классифицировать текст по различным категориям, таким как спам/не спам, положительный/отрицательный отзыв и т.д.
6. **Разговор**: Мы можем вести разговор с пользователями, отвечая на их вопросы и комментарии в 

In [ ]:
# === ЗАГРУЗКА / ИНИЦИАЛИЗАЦИЯ КАТЕГОРИЙ ===
def load_categories():
    if os.path.exists(CATEGORIES_FILE):
        with open(CATEGORIES_FILE, 'r', encoding='utf-8') as f:
            return json.load(f)
    else:
        print("📁 Создаём начальный список категорий...")
        save_categories(DEFAULT_CATEGORIES)
        return DEFAULT_CATEGORIES.copy()

def save_categories(categories):
    with open(CATEGORIES_FILE, 'w', encoding='utf-8') as f:
        json.dump(sorted(list(set(categories))), f, ensure_ascii=False, indent=2)

In [ ]:
# === НАСТРОЙКИ (с чекпойнтами, адаптивность выключена) ===
TEXT_COLUMN = "review_text"

MAX_BATCH = 180
SELECTION_LEN = 300
CATEGORIES_FILE = "current_categories.json"  # Файл для хранения списка категорий
CHECKPOINT_DIR = "checkpoints"

# Инициализация начальных категорий
DEFAULT_CATEGORIES = [
    "Карты",
    "Банкоматы",
    "Кэшбэк / Бонусы",
    "Обслуживание в офисе",
    "Вклады",
    "Кредиты",
    "Курьерская служба",
    "Приложение / сайт",
    "Служба поддержки",
    "Счета",
    "Прочие услуги"
]
SENTIMENTS = [
   "позитив", "негатив", "нейтральный"
]


# === ПРОМПТ СО СТАТИЧНЫМИ ===
def build_system_prompt(allowed_categories):
    categories_list = ', '.join([f'"{cat}"' for cat in allowed_categories])
    return f"""
Проанализируй отзыв клиента банка и определи, какие продукты или услуги упоминаются, а также тональность отзыва (позитив/негатив/нейтральный)

Правила:
1. Категория должна быть ТОЛЬКО из следующего списка: {categories_list} . Запрещено выдумывать новые.
Запрещено добавлять категории, если они не упоминаются в отзыве
Дополнительное правило для определения категорий взаимодействия с клиентом
**признаки обслуживания в офисе**
- упоминания что КЛИЕНТ пришел, приехал в банк, в отделение, в офис
**признаки курьерской службы**
- упоминание что ПРЕДСТАВИТЕЛЬ БАНКА пришел, приехал к клиенту, а не наоборот. упоминание назначения времени и места встречи
**признаки службы поддержки**
- упоминание что клиент звонил, писал в чат, на сайт, общался с ботом

2. Категория "Прочие услуги" применяется ТОЛЬКО если ни одна из прочих категорий не подходит.
  Общие формулировки вроде "отличный банк"/"ужасный банк" без указания обстоятельств не классифицируем.
3. Тональность должна иметь ТОЛЬКО одно из трех значений: позитив, негатив или нейтральный.
  Если по одной категории найдено несколько смысловых фрагментов, то категория не повторяется, а сентименты складываются
  Например, "условия по картам хорошие, но цифры с пластика быстро стираются". -1 +1 = 0 -> "нейтрально"
4. Саммари должно содержать аспект и отношение к нему клиента. Например "клиент не смог восстановить доступ к заблокированному счету"
  Не делай саммари вроде "Отзыв о карте Газпромбанка" - это не информативно.


Формат ответа:

  {{ "annotations": [{{"category": "...", "summary": "...", "sentiment": "..." }}, ...] }}

Ответ должен быть ТОЛЬКО валидным JSON.
"""

# === ОБРАБОТКА ОТЗЫВА ЧЕРЕЗ LLM ===
def call_llm_with_categories(client, model, review, categories):
    prompt = build_system_prompt(categories)
    raw_response = call_llm_for_review(client, model, review, prompt)
    return clean_and_parse_json(raw_response) if raw_response else None

# === ВЫПОЛНЕНИЕ ===
try:
    # 0. Загружаем файл
    filepath = upload_csv()

    # 1. Читаем CSV
    df = load_csv_to_dataframe(filepath)


    # 2. Извлекаем отзывы
    reviews = extract_reviews(df, text_column=TEXT_COLUMN)
    total_reviews = len(reviews)
    print(f"📊 Всего отзывов для обработки: {total_reviews}")

    if total_reviews == 0:
        raise ValueError("Список отзывов пуст. Проверьте название столбца.")

    # 3. Загружаем текущие категории
    current_categories = load_categories()
    print(f"🏷️  Текущие категории ({len(current_categories)}): {current_categories}")

    # 4. Обработка порциями
    all_results = []
    checkpoint_counter = 0

    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

    for i in range(0, total_reviews, MAX_BATCH):
        batch = reviews[i:i + MAX_BATCH]
        batch_results = []

        print(f"\n--- Обработка батча {i//MAX_BATCH + 1} ({i}–{min(i + MAX_BATCH, total_reviews)}) ---")
        print(f"📌 Используется {len(current_categories)} категорий")

        new_categories_in_batch = []  # собираем новые категории из этого батча

        for idx_in_batch, review in enumerate(batch):
            global_idx = i + idx_in_batch

            print(f"[{global_idx + 1}/{total_reviews}] Обработка отзыва...")
            response = call_llm_with_categories(client, model, review, current_categories)

            if not response:
                entry = build_final_entry(global_idx, None)
                batch_results.append(entry)
                continue

            # Проверяем, есть ли новая категория
            if "new_category" in response:
                new_cat = response["new_category"].strip()
                if new_cat and new_cat not in current_categories and new_cat not in new_categories_in_batch:
                    new_categories_in_batch.append(new_cat)
                    print(f"✨ Новая категория: '{new_cat}'")

                # Удаляем new_category из финального вывода
                response.pop("new_category", None)

            entry = build_final_entry(global_idx, response)
            batch_results.append(entry)

        # После обработки батча — обновляем глобальный список категорий
        if new_categories_in_batch:
            current_categories.extend(new_categories_in_batch)
            current_categories = sorted(list(set(current_categories)))  # dedup
            save_categories(current_categories)
            print(f"✅ Добавлено новых категорий: {len(new_categories_in_batch)}. Всего категорий: {len(current_categories)}")

        # Сохраняем результаты батча
        all_results.extend(batch_results)
        save_checkpoint(batch_results, checkpoint_counter, prefix="batch_")
        checkpoint_counter += 1

    # 5. Финальное сохранение
    #all_results_flat = flatten_results(all_results)   #убираем лишний ключ 'results'
    final_filename = "llm_annotations_full.json"
    with open(final_filename, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"🎉 Обработка завершена! Сохранено {len(all_results)} записей.")
    print(f"📌 Итоговое количество категорий: {len(current_categories)}")
    files.download(final_filename)
    files.download(CATEGORIES_FILE)

except Exception as e:
    print(f"🚨 Ошибка: {e}")
    print("💡 Совет: используйте merge_checkpoints(), чтобы восстановить данные.")

Загрузите CSV-файл...


Saving data_for_mark_330.csv to data_for_mark_330.csv
Файл 'data_for_mark_330.csv' успешно загружен.
Файл успешно прочитан. Размер: (330, 3)
Извлечено 330 отзывов из столбца 'review_text'.
📊 Всего отзывов для обработки: 330
📁 Создаём начальный список категорий...
🏷️  Текущие категории (11): ['Карты', 'Банкоматы', 'Кэшбэк / Бонусы', 'Обслуживание в офисе', 'Вклады', 'Кредиты', 'Курьерская служба', 'Приложение / сайт', 'Служба поддержки', 'Счета', 'Прочие услуги']

--- Обработка батча 1 (0–180) ---
📌 Используется 11 категорий
[1/330] Обработка отзыва...
[2/330] Обработка отзыва...
[3/330] Обработка отзыва...
[4/330] Обработка отзыва...
[5/330] Обработка отзыва...
[6/330] Обработка отзыва...
[7/330] Обработка отзыва...
[8/330] Обработка отзыва...
[9/330] Обработка отзыва...
[10/330] Обработка отзыва...
[11/330] Обработка отзыва...
[12/330] Обработка отзыва...
[13/330] Обработка отзыва...
[14/330] Обработка отзыва...
[15/330] Обработка отзыва...
[16/330] Обработка отзыва...
[17/330] Обрабо

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# загрузить финальный результат
import json
import pandas as pd
from collections import Counter

# Загружаем результаты
with open('llm_annotations_full.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# извлечение категорий
from collections import Counter
import pandas as pd

categories = []
for item in data:

    annotations = item["results"].get("annotations", [])
    for ann in annotations:
        cat = ann.get("category", "").strip()
        if cat:
            categories.append(cat)

print(f"✅ Найдено упоминаний: {len(categories)}")
print(f"📌 Уникальных категорий: {len(set(categories))}")
category_counts = Counter(categories)

# Превращаем в обычный словарь (если нужно)
category_dict = dict(category_counts)

# Создаём DataFrame
df_counts = pd.DataFrame(list(category_counts.items()), columns=['category', 'count'])

# Сортируем по убыванию частоты
df_counts = df_counts.sort_values(by='count', ascending=False).reset_index(drop=True)



✅ Найдено упоминаний: 847
📌 Уникальных категорий: 12


In [ ]:

df_counts.to_csv('category_frequency.csv', index=False, encoding='utf-8-sig')
print("💾 Таблица сохранена: category_frequency.csv")
files.download('category_frequency.csv')

💾 Таблица сохранена: category_frequency.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# дополняем исходный файл категориями и саммари
df = load_csv_to_dataframe(filepath)

# Загружаем результаты LLM
with open('llm_annotations_full.json', 'r', encoding='utf-8') as f:
    annotations_data = json.load(f)

Файл успешно прочитан. Размер: (330, 3)


In [ ]:
# Подготовим список для новых данных
expanded_rows = []

for item in annotations_data:
    review_id = item["review_id"]
    annotations = item["results"].get("annotations", [])

    # Если аннотаций нет — всё равно создаём строку с пустыми значениями
    if not annotations:
        expanded_rows.append({
            "review_id": review_id,
            "category": None,
            "summary": None
        })
    else:
        for ann in annotations:
            expanded_rows.append({
                "review_id": review_id,
                "category": ann["category"],
                "summary": ann["summary"]
            })

# Создаём DataFrame с аннотациями
df_annotations = pd.DataFrame(expanded_rows)

In [ ]:
# Убедимся, что df имеет правильный индекс
df_reset = df.reset_index(drop=True)  # теперь индекс: 0, 1, 2, ..., N-1

# === ВАРИАНТ 1: ОДНА СТРОКА НА ОТЗЫВ, списки категорий и саммари ===
grouped = df_annotations.groupby('review_id').agg(
    extracted_categories=('category', list),
    extracted_summaries=('summary', list)
)
# Оставляем review_id как индекс

# Присоединяем по индексу: review_id == индексу df_reset
df_enriched = df_reset.join(grouped, how='left')

# Заполняем NaN пустыми списками
df_enriched['extracted_categories'] = df_enriched['extracted_categories'].apply(lambda x: x if isinstance(x, list) else [])
df_enriched['extracted_summaries'] = df_enriched['extracted_summaries'].apply(lambda x: x if isinstance(x, list) else [])

In [ ]:
# сохраняем результаты
output_filename = "enriched_bank_reviews.csv"
df_enriched.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"✅ Обогащённый CSV сохранён: {output_filename}")
files.download(output_filename)

✅ Обогащённый CSV сохранён: enriched_bank_reviews.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Загружаем аннотации
with open('llm_annotations_full.json', 'r', encoding='utf-8') as f:
    annotations = json.load(f)

# Исходный DataFrame (уже загружен ранее)
# df — должен быть без изменений порядка строк
df_reset = df.reset_index(drop=True)  # гарантируем обычный индекс 0,1,2...

In [ ]:
# Колонки, которые хочешь добавить к аннотациям
COLUMNS_TO_INCLUDE = [TEXT_COLUMN, 'review_dttm', 'review_text']  # например

# Проверим, есть ли нужные колонки
available_cols = [col for col in COLUMNS_TO_INCLUDE if col in df_reset.columns]
if not available_cols:
    print("⚠️  Нет нужных колонок. Добавим только текст.")
    available_cols = [TEXT_COLUMN] if TEXT_COLUMN in df_reset.columns else []

# Преобразуем df в словарь по индексу
metadata_dict = df_reset[available_cols].to_dict('index')  # {0: {...}, 1: {...}, ...}

# Обновляем аннотации
enriched_annotations = []

for item in annotations:
    review_id = item["review_id"]

    # Проверяем, что такой ID существует
    if review_id >= len(df_reset):
        print(f"⚠️  Пропущено: review_id={review_id} вне диапазона (всего {len(df_reset)} строк)")
        continue

    # Достаём данные из df
    row_data = metadata_dict.get(review_id, {})

    # Формируем обогащённую запись
    enriched_item = {
        "review_id": review_id,
    }

    # Добавляем выбранные колонки
    for col in available_cols:
        value = row_data.get(col, None)
        # Приводим к простым типам (на всякий случай)
        enriched_item[col] = str(value) if pd.isna(value) else value

    # Добавляем результаты LLM
    enriched_item["results"] = item.get("results", {})

    enriched_annotations.append(enriched_item)


/tmp/ipython-input-3155780198.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  metadata_dict = df_reset[available_cols].to_dict('index')  # {0: {...}, 1: {...}, ...}


In [ ]:
output_filename = "llm_annotations_enriched.json"

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(enriched_annotations, f, ensure_ascii=False, indent=2)

print(f"✅ Обогащённый JSON сохранён: {output_filename}")
files.download(output_filename)

✅ Обогащённый JSON сохранён: llm_annotations_enriched.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>